In [1]:
import numpy as np
import numpy.typing as npt

***This example is obtained from Kenneth J. Beers. Numerical methods for chemical engineering. Cambridge University Press, Cambridge, UK \[u.a.\], repr. edition, 2009. Chapter 8 Bayesian statistics and parameter estimation.***

We consider a batch reactor hosting a reaction $ \nu_A A+ \nu_B B -> \nu_C C$. We assume the balances volume, i.e., the reactor volume, to be isothermal ($T$=const).

The volumetric reaction rate expression is thus $r_{R1}=k_1 c_A^{\nu_A} c_B^{\nu_B}$. The initial concentrations of components $A$ and $B$ are known a priori.

We are interested in determining the stoichiometric coefficients $\nu_A$ and $\nu_B$, as well as $k_1(T)=k_1$.

For this, we exploit the relation

$\frac{dc_C}{dt}|_{t=0}=r_{R1}=k_1 [c_A(0)]^{\nu_A} [c_B(0)]^{\nu_B}$.

We run experiments with known initial concentrations of $A$ and $B$, and measure the slope of $c_C(t=0)$ and thus obtain measurements of $r_{R1}$. The measurements are given below:

In [2]:
c_A_measurements = np.array([0.1, 0.2, 0.1, 0.2, 0.05, 0.2])
c_B_measurements = np.array([0.1, 0.1, 0.2, 0.2, 0.2, 0.05])
r_R1_measurements = np.array([0.0246, 0.0483, 0.0501, 0.1003, 0.0239, 0.0262])*1E-03

We want to find $k_1, \nu_A, \nu_B$ using linear regression. For this, we first need to define a linear model by applying the base-10 logarithm such that

$log_{10}(r_{R1})= log_{10}(k_1)+ \nu_A \cdot  log_{10}(c_A(0))+\nu_B \cdot log_{10}(c_B(0))$.

We consider the inputs of our linear regression problem to be $\boldsymbol{x}=[log_{10}(c_A(0)), log_{10}(c_B(0))]$. The output is $y=r_{R1}$. The set of parameters is $\boldsymbol{\theta}=[log_{10}(k_1), \nu_A, \nu_B]$ such that we can derive

$y=\theta_0+\theta_1x_1+\theta_2x_2$.

In [3]:
def linear_regression(x_feature: npt.NDArray, y_target:npt.NDArray) -> npt.NDArray:

    # Add a column of ones to X for the bias term (intercept)
    X_b = np.column_stack((np.ones((x_feature.shape[0], 1)), x_feature))

    # Solve the normal equation to find the coefficients (theta)
    theta = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y_target)

    return theta

In [ ]:
x_feature = np.vstack((np.log10(c_A_measurements), np.log10(c_B_measurements))).T
y_target = np.log10(r_R1_measurements)
theta = linear_regression(x_feature, y_target)
print(f"k_1: {10**theta[0]}")
print('v_A: ', theta[1])
print('v_B: ', theta[2])

Now, we want to find $k_1, \nu_A, \nu_B$ using nonlinear regression.

$\frac{dc_C}{dt}|_{t=0}=r_{R1}=k_1 [c_A(0)]^{\nu_A} [c_B(0)]^{\nu_B}$.

We consider the inputs of our nonlinear regression problem to be $\boldsymbol{\tilde{x}}=[c_A(0), c_B(0)]$. The output is $\tilde{y}=r_{R1}$. The set of parameters is $\boldsymbol{\tilde{\theta}}=[k_1, \nu_A, \nu_B]$ such that we can derive

$y=\tilde{\theta}_0x_1^{\tilde{\theta}_1}x_2^{\tilde{\theta}_2}$.

NOTE: To use [scipy.optimize.curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html), the model function `f(x,*params)` must take the independent variable `x` as the first argument and the parameters to fit as separate remaining arguments!

In [5]:
def rate_expression(x:npt.NDArray, k1:float, nuA:float, nuB:float) -> npt.NDArray:
    """
    Function to express the reaction rate for an irreversible reaction with reagents A and B 
    with reaction rate constant k1: A+B ->[k1] ...
    Parameters
    ----------
        x (npt.NDArray): initial concentration measurements for species A and B
        k1 (float): reaction rate constant
        nuA (float): stoichiometric coefficient of species A
        nuB (float): stoichiometric coefficient of species B
    Returns
    -------
        (npt.NDArray): reaction rate
    """

    return k1*x[0]**nuA*x[1]**nuB


Now, let's apply scipy.optimize.curve_fit!

In [ ]:
from scipy.optimize import curve_fit

popt, pcov = curve_fit(f=rate_expression, xdata=[c_A_measurements, c_B_measurements], ydata=r_R1_measurements)
print("k_1: ", popt[0])
print('v_A: ', popt[1])
print('v_B: ', popt[2])